In [1]:
import pytorch_lightning as L

from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
    ModelSummary,
)

import torch

from model.deepfont import DeepFont
from model.lightning_generate_callback import GenerateCallback
from model.lightning_wrappers import DeepFontAutoencoderWrapper, DeepFontWrapper

from dataset.dataset import FontWeightDataset, FWDataset
from dataset.transformations import *

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from model.configs import load_config
import numpy as np
import random 
import os 
import cv2 as cv


In [2]:
config = load_config("F:/FWC216/configs/adobe-vfr.yaml")
#config

In [3]:
# Load raw data
data_path = "data/"
folders = os.listdir(data_path)

img_paths = []

for folder in folders: 
    cpath = data_path + folder 

    images = os.listdir(cpath)
    for image in images: 
        a_path = "F:/FWC216/" + data_path + folder + "/" + image
        img_paths.append(a_path)
        
    

In [4]:
# z = 0
# for i in img_paths: 
#     print(i)
#     img = cv.imread(i, cv.IMREAD_GRAYSCALE)
#     np_img = np.array(img, dtype=np.uint8)
#     z+=1
#     print(z)
#     print("------------------------")

In [5]:
# Shuffle and split data

random.shuffle(img_paths) 

total_size = len(img_paths)
train_size = int(0.7*total_size)
test_size  = int(0.1*total_size) 
valid_size =  total_size - train_size - test_size

train_data = img_paths[:train_size]
test_data  = img_paths[train_size:train_size + test_size]
valid_data = img_paths[train_size + test_size:]

In [6]:
# t = T1()
# for path in train_data: 
#     img = cv.imread(path, cv.IMREAD_GRAYSCALE)
#     img1 = Ptrans(img)
#     img2 = t(image=img)['image']
#     print(img1.shape, img2.shape)
#     break

    


In [7]:
# t_img.shape
# cv.imshow("n", t_img)


In [8]:
# Torch dataset


train_dataset =  FWDataset(img_folder = train_data)
test_dataset  =  FWDataset(img_folder = test_data )
valid_dataset =  FWDataset(img_folder = valid_data)



In [9]:
# Data loader 
batch_size = 64
train_loader =  DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers= 5, persistent_workers= True)

valid_loader =  DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers= 5, persistent_workers= True)

test_loader  =  DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers= 5, persistent_workers= True)

In [10]:
torch.set_float32_matmul_precision("high")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")



Device: cuda


In [11]:
# Define model
model = DeepFontAutoencoderWrapper()
model.to(device)

DeepFontAutoencoderWrapper(
  (autoencoder): DeepFontAutoencoder(
    (encoder): Sequential(
      (0): Conv2d(1, 64, kernel_size=(12, 12), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ReLU()
      (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (decoder): Sequential(
      (0): Upsample(scale_factor=2.0, mode='bilinear')
      (1): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): ReLU()
      (3): Upsample(scale_factor=2.0, mode='bilinear')
      (4): ConvTranspose2d(64, 1, kernel_size=(12, 12), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  

In [12]:
torch.cuda.empty_cache()

In [13]:
# Define trainer 

training_callbacks = [
        EarlyStopping(monitor="val_loss", mode="min"),
        LearningRateMonitor(logging_interval="step"),
        ModelCheckpoint(
            dirpath="./output_ae",
            save_top_k=2,
            monitor="val_loss",
            filename="deepfont-{epoch:02d}-{val_loss:.4f}-{val_accuracy:.4f}",
            save_last=True,
        ),
        ModelSummary(-1),
    ]



sample_index = 10 % len(train_dataset)
training_callbacks.append(
        GenerateCallback(
            train_dataset[sample_index][0].unsqueeze(0), every_n_epochs=5
        ),
)

trainer = L.Trainer(
        # If ever doing gradient clipping here, remember that this
        # has an effect on training: it will take likely twice as
        # long to get to the same accuracy!
        # gradient_clip_algorithm="norm",
        # gradient_clip_val=1.0,
    max_epochs=40,
    callbacks=training_callbacks,
)


trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=valid_loader,
    ckpt_path= None,
)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Missing logger folder: f:\FWC216\lightning_logs
f:\FWC\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory F:\FWC216\output_ae exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                  | Type                | Params | Mode  | In sizes         | Out sizes       
-------------------------------------------------------------------------------------------------------------
0  | autoencoder           | DeepFontAutoencoder | 166 K  | train | [2, 1, 105, 105] | [2, 1, 105, 105]
1  | autoencoder.encoder   | Sequential          | 83.5 K | train | [2, 1, 105, 105] | [2, 128, 12, 12]
2  | autoencoder.encoder.0 | Conv2d              |

Epoch 18: 100%|██████████| 438/438 [00:32<00:00, 13.49it/s, v_num=0, train_loss=0.000797, val_loss=0.000868]


In [14]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 63/63 [00:07<00:00,  7.89it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.0008794337045401335
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0008794337045401335}]